# Advanced Analysis: Trader Behavior vs Market Sentiment
**Continuation of Web3 Trading Analysis**

In [ ]:
# Import libraries and load preprocessed data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

# Load and preprocess data (assuming data is already cleaned from notebook_1)
fear_greed = pd.read_csv('csv_files/fear_greed_index.csv')
trading_data = pd.read_csv('csv_files/historical_data.csv')

# Data preprocessing
fear_greed['date'] = pd.to_datetime(fear_greed['date'])
trading_data['Timestamp IST'] = pd.to_datetime(trading_data['Timestamp IST'], format='%d-%m-%Y %H:%M')
trading_data['Date'] = trading_data['Timestamp IST'].dt.date
trading_data['Date'] = pd.to_datetime(trading_data['Date'])

numeric_cols = ['Execution Price', 'Size Tokens', 'Size USD', 'Closed PnL', 'Fee']
for col in numeric_cols:
    trading_data[col] = pd.to_numeric(trading_data[col], errors='coerce')

## 3. Exploratory Data Analysis

In [ ]:
# Fear & Greed Index distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Distribution of Fear & Greed values
axes[0,0].hist(fear_greed['value'], bins=30, alpha=0.7, color='skyblue')
axes[0,0].set_title('Distribution of Fear & Greed Index Values')
axes[0,0].set_xlabel('Index Value')
axes[0,0].set_ylabel('Frequency')

# Classification counts
fear_greed['classification'].value_counts().plot(kind='bar', ax=axes[0,1], color='lightcoral')
axes[0,1].set_title('Fear & Greed Classifications')
axes[0,1].set_xlabel('Classification')
axes[0,1].set_ylabel('Count')
axes[0,1].tick_params(axis='x', rotation=45)

# Time series of Fear & Greed Index
axes[1,0].plot(fear_greed['date'], fear_greed['value'], alpha=0.7, color='green')
axes[1,0].set_title('Fear & Greed Index Over Time')
axes[1,0].set_xlabel('Date')
axes[1,0].set_ylabel('Index Value')
axes[1,0].tick_params(axis='x', rotation=45)

# Trading volume over time
daily_volume = trading_data.groupby('Date')['Size USD'].sum().reset_index()
axes[1,1].plot(daily_volume['Date'], daily_volume['Size USD'], alpha=0.7, color='orange')
axes[1,1].set_title('Daily Trading Volume (USD)')
axes[1,1].set_xlabel('Date')
axes[1,1].set_ylabel('Volume (USD)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('outputs/eda_overview.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Merging Datasets and Correlation Analysis

In [ ]:
# Merge datasets on date
# First, aggregate trading data by date
daily_trading_stats = trading_data.groupby('Date').agg({
    'Size USD': ['sum', 'mean', 'count'],
    'Closed PnL': ['sum', 'mean'],
    'Account': 'nunique',
    'Execution Price': 'mean'
}).round(2)

# Flatten column names
daily_trading_stats.columns = ['_'.join(col).strip() for col in daily_trading_stats.columns]
daily_trading_stats = daily_trading_stats.reset_index()

# Rename columns for clarity
daily_trading_stats.columns = ['Date', 'Total_Volume_USD', 'Avg_Trade_Size_USD', 'Trade_Count', 
                              'Total_PnL', 'Avg_PnL', 'Unique_Traders', 'Avg_Price']

# Merge with fear & greed data
merged_data = pd.merge(fear_greed, daily_trading_stats, left_on='date', right_on='Date', how='inner')

print(f"Merged dataset shape: {merged_data.shape}")
print(f"Date range: {merged_data['date'].min()} to {merged_data['date'].max()}")
display(merged_data.head())

In [ ]:
# Correlation analysis
correlation_cols = ['value', 'Total_Volume_USD', 'Avg_Trade_Size_USD', 'Trade_Count', 
                   'Total_PnL', 'Avg_PnL', 'Unique_Traders', 'Avg_Price']

correlation_matrix = merged_data[correlation_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='RdYlBu_r', center=0, 
            square=True, fmt='.3f', cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix: Market Sentiment vs Trading Metrics', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig('outputs/correlation_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Print key correlations
print("Key Correlations with Fear & Greed Index:")
for col in correlation_cols[1:]:
    corr = correlation_matrix.loc['value', col]
    print(f"{col}: {corr:.3f}")

## 5. Sentiment-Based Trading Behavior Analysis

In [ ]:
# Analyze trading behavior by sentiment categories
sentiment_analysis = merged_data.groupby('classification').agg({
    'Total_Volume_USD': ['mean', 'std'],
    'Trade_Count': ['mean', 'std'],
    'Total_PnL': ['mean', 'std'],
    'Unique_Traders': ['mean', 'std'],
    'value': ['mean', 'min', 'max']
}).round(2)

print("Trading Behavior by Market Sentiment:")
display(sentiment_analysis)

In [ ]:
# Visualize trading patterns by sentiment
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Volume by sentiment
sns.boxplot(data=merged_data, x='classification', y='Total_Volume_USD', ax=axes[0,0])
axes[0,0].set_title('Trading Volume by Market Sentiment')
axes[0,0].tick_params(axis='x', rotation=45)

# Trade count by sentiment
sns.boxplot(data=merged_data, x='classification', y='Trade_Count', ax=axes[0,1])
axes[0,1].set_title('Number of Trades by Market Sentiment')
axes[0,1].tick_params(axis='x', rotation=45)

# PnL by sentiment
sns.boxplot(data=merged_data, x='classification', y='Total_PnL', ax=axes[0,2])
axes[0,2].set_title('Total PnL by Market Sentiment')
axes[0,2].tick_params(axis='x', rotation=45)

# Unique traders by sentiment
sns.boxplot(data=merged_data, x='classification', y='Unique_Traders', ax=axes[1,0])
axes[1,0].set_title('Unique Traders by Market Sentiment')
axes[1,0].tick_params(axis='x', rotation=45)

# Average trade size by sentiment
sns.boxplot(data=merged_data, x='classification', y='Avg_Trade_Size_USD', ax=axes[1,1])
axes[1,1].set_title('Average Trade Size by Market Sentiment')
axes[1,1].tick_params(axis='x', rotation=45)

# Average PnL by sentiment
sns.boxplot(data=merged_data, x='classification', y='Avg_PnL', ax=axes[1,2])
axes[1,2].set_title('Average PnL by Market Sentiment')
axes[1,2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('outputs/sentiment_trading_patterns.png', dpi=300, bbox_inches='tight')
plt.show()